In [1]:
!pip install annoy

     |████████████████████████████████| 655kB 5.4MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391588 sha256=babbe5670883a40e10decff7397d9088653153401d43b5359215aa8112e00d49
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [3]:
from annoy import AnnoyIndex
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
anime = pd.read_csv('/content/anime.csv', index_col='Id')

In [5]:
sample_submit = pd.read_csv('/content/sample_submission.csv')

In [46]:
user_ratings = pd.read_csv('/content/user_ratings.csv')
user_ratings = user_ratings.loc[user_ratings["rating"] != 0]

In [48]:
user_ratings

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,121,8,1,0
1,0,12815,10,1,3
2,0,3588,9,1,13
3,0,392,8,3,0
4,1,1575,8,2,25
...,...,...,...,...,...
19082564,98598,355,7,2,24
19082565,98598,23273,9,2,22
19082566,98598,34902,7,2,12
19082567,98598,14813,8,2,13


In [8]:
anime

,Name,Avg score,Genres,English name,Type,Num episodes,When aired,Date of premiere,Producers,Licensors,Studios,Based on,Duration,Age limit,In list,Group members,In favourites,Watching,Finished,On hold,Dropped,Planning to watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),39,1251960,61971,105808,718161,71513,26678,329800,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),518,273145,1174,4143,208333,1935,770,57964,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min. per ep.,PG-13 - Teens 13 or older,201,558913,12944,29113,343492,25465,13925,146918,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...","Funimation, Bandai Entertainment",Sunrise,Original,25 min. per ep.,PG-13 - Teens 13 or older,1467,94683,587,4300,46165,5121,5378,33719,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,"TV Tokyo, Dentsu",Unknown,Toei Animation,Manga,23 min. per ep.,PG - Children,4369,13224,18,642,7314,766,1108,3394,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",Unknown,ONA,Unknown,"Apr 4, 2021 to ?",Unknown,Unknown,Unknown,Unknown,Novel,Unknown,Unknown,13116,354,6,0,0,0,0,354,Unknown,Unknown,Unknown,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",Unknown,TV,Unknown,2021 to ?,Unknown,Kadokawa,Unknown,Passione,Manga,Unknown,Unknown,17562,7010,70,14,0,14,0,6982,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry – SOTSU,TV,Unknown,"Jul, 2021 to ?",Summer 2021,Unknown,Unknown,Unknown,Visual novel,Unknown,R - 17+ (violence & profanity),17558,11309,48,3,0,0,0,11306,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [45]:
set(anime.Studios.values)

{'10Gauge',
 '10Gauge, Studio DURIAN',
 '1IN',
 '2:10 AM Animation',
 '33 Collective',
 '8bit',
 'A-1 Pictures',
 'A-1 Pictures, Bridge',
 'A-1 Pictures, Bridge, CloverWorks',
 'A-1 Pictures, Lapin Track',
 'A-1 Pictures, Ordet',
 'A-1 Pictures, TROYCA',
 'A-1 Pictures, Trigger, CloverWorks',
 'A-Line',
 'A-Real',
 'A.C.G.T.',
 'ACC Production',
 'AIC',
 'AIC ASTA',
 'AIC Build',
 'AIC Classic',
 'AIC Frontier',
 'AIC PLUS+',
 'AIC Spirits',
 'AIC Spirits, Asread',
 'AIC Spirits, BeSTACK',
 'AIC Spirits, Digital Frontier',
 'AIC Spirits, Group TAC',
 'AIC Takarazuka',
 'AIC, APPP',
 'AIC, Artmic',
 'AIC, Artmic, Darts',
 'AIC, Artmic, animate Film',
 'AIC, Darts',
 'AIC, Magic Bus, Ashi Production',
 'AIC, Studio Kyuuma',
 'AIC, animate Film',
 'ANIDO FILM',
 'APPP',
 'AQUA ARIS',
 'ASK Animation Studio',
 'AXsiZ',
 'Academy Productions',
 'Acca effe, Giga Production',
 'Actas',
 'Actas, Bee Media',
 "Actas, Brain's Base",
 'Actas, Production Reed',
 'Actas, Studio 3Hz',
 'Actas, Syner

In [10]:
anime.loc[48492]

Name                     Scarlet Nexus
Avg score                      Unknown
Genres                 Action, Fantasy
English name                   Unknown
Type                                TV
Num episodes                   Unknown
When aired              Jul, 2021 to ?
Date of premiere           Summer 2021
Producers            Bandai Namco Arts
Licensors                      Unknown
Studios                        Sunrise
Based on                          Game
Duration                       Unknown
Age limit                      Unknown
In list                          17563
Group members                      310
In favourites                        0
Watching                             4
Finished                             0
On hold                              1
Dropped                              1
Planning to watch                  304
Score-10                       Unknown
Score-9                        Unknown
Score-8                        Unknown
Score-7                  

In [51]:
def create_vector(df, anime):
    v = []
    ratings = []
    studios = list(set(anime.Studios.values))
    for i in range(len(df)):
        print(i)
        avg_score = anime.loc[df['anime_id'].iloc[i]]['Avg score']
        num_episodes = anime.loc[df['anime_id'].iloc[i]]['Num episodes']
        in_list = anime.loc[df['anime_id'].iloc[i]]['In list']
        studio = studios.index(anime.loc[df['anime_id'].iloc[i]]['Studios'])
        if avg_score == 'Unknown':
            avg_score = 0
        if num_episodes == 'Unknown':
            num_episodes = 0
        if in_list == 'Unknown':
            in_list = 0
        v.append([df['user_id'].iloc[i],
                     df['anime_id'].iloc[i],
                     float(avg_score),
                     float(num_episodes),
                    float(in_list),
                    studio
        ])
        
        ratings.append(df['rating'].iloc[i])
        if i == 100000:
            break
    return v, ratings

In [52]:
v, ratings = create_vector(user_ratings, anime)

Выходные данные были обрезаны до нескольких последних строк (5000).
95001
95002
95003
95004
95005
95006
95007
95008
95009
95010
95011
95012
95013
95014
95015
95016
95017
95018
95019
95020
95021
95022
95023
95024
95025
95026
95027
95028
95029
95030
95031
95032
95033
95034
95035
95036
95037
95038
95039
95040
95041
95042
95043
95044
95045
95046
95047
95048
95049
95050
95051
95052
95053
95054
95055
95056
95057
95058
95059
95060
95061
95062
95063
95064
95065
95066
95067
95068
95069
95070
95071
95072
95073
95074
95075
95076
95077
95078
95079
95080
95081
95082
95083
95084
95085
95086
95087
95088
95089
95090
95091
95092
95093
95094
95095
95096
95097
95098
95099
95100
95101
95102
95103
95104
95105
95106
95107
95108
95109
95110
95111
95112
95113
95114
95115
95116
95117
95118
95119
95120
95121
95122
95123
95124
95125
95126
95127
95128
95129
95130
95131
95132
95133
95134
95135
95136
95137
95138
95139
95140
95141
95142
95143
95144
95145
95146
95147
95148
95149
95150
95151
95152
95153
95154
95155
95

In [53]:
X_train, X_test, y_train, y_test = train_test_split(v, ratings, test_size=0.1, random_state=1325642)

In [54]:
t = AnnoyIndex(6, 'euclidean')  # Length of item vector that will be indexed
for i in range(len(X_train)):  
    t.add_item(i, X_train[i])
t.build(10) # 10 trees
t.save('test.ann')

# ...


True

In [55]:
def eval(u, X_test, y_test, y_train):
    preds = []
    for i in range(len(X_test)):
        print(i)
        n = u.get_nns_by_vector(X_test[i], 10)
        preds.append((y_train[n[0]] + y_train[n[1]] + y_train[n[2]] + y_train[n[3]] + y_train[n[4]] + y_train[n[5]] + y_train[n[6]] + y_train[n[7]] + y_train[n[8]] + y_train[n[9]]) / 10)
    return preds

In [56]:
anime.loc[249]

Name                                                          InuYasha
Avg score                                                         7.85
Genres               Action, Adventure, Comedy, Historical, Demons,...
English name                                                  InuYasha
Type                                                                TV
Num episodes                                                       167
When aired                                Oct 16, 2000 to Sep 13, 2004
Date of premiere                                             Fall 2000
Producers                              Yomiuri Telecasting, Shogakukan
Licensors                                                    VIZ Media
Studios                                                        Sunrise
Based on                                                         Manga
Duration                                               24 min. per ep.
Age limit                                    PG-13 - Teens 13 or older
In lis

In [57]:
sample_submit

,Id,rating
0,0 249,7
1,0 4181,4
2,0 813,3
3,0 2167,6
4,1 1838,3
...,...,...
876524,98598 15687,8
876525,98598 1535,5
876526,98598 37520,8
876527,98598 35222,5


In [58]:

u = AnnoyIndex(6, 'euclidean')
u.load('test.ann') # super fast, will just mmap the file
#print(u.get_nns_by_vector([0, 249, 7.85, 167], 5))

True

In [59]:
preds = eval(u, X_test, y_test, y_train)

Выходные данные были обрезаны до нескольких последних строк (5000).
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
51

In [60]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, preds)

2.5919928007199284

In [ ]:
y_test

[0,
 0,
 9,
 7,
 5,
 7,
 7,
 7,
 0,
 7,
 9,
 6,
 10,
 0,
 10,
 0,
 0,
 5,
 0,
 7,
 8,
 8,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 9,
 0,
 6,
 10,
 5,
 8,
 5,
 9,
 6,
 0,
 0,
 0,
 0,
 7,
 7,
 0,
 0,
 7,
 0,
 10,
 7,
 9,
 4,
 0,
 8,
 0,
 0,
 8,
 0,
 0,
 7,
 8,
 9,
 5,
 7,
 7,
 7,
 0,
 6,
 9,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 6,
 0,
 8,
 9,
 6,
 8,
 6,
 0,
 7,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 7,
 10,
 9,
 0,
 0,
 7,
 2,
 8,
 0,
 0,
 10,
 0,
 5,
 7,
 8,
 2,
 0,
 7,
 6,
 8,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 9,
 0,
 0,
 7,
 10,
 0,
 8,
 0,
 0,
 10,
 0,
 0,
 9,
 9,
 0,
 9,
 0,
 9,
 7,
 0,
 0,
 0,
 9,
 6,
 8,
 9,
 7,
 7,
 0,
 8,
 0,
 0,
 0,
 7,
 7,
 0,
 0,
 5,
 0,
 6,
 9,
 10,
 8,
 8,
 0,
 9,
 0,
 6,
 0,
 9,
 0,
 0,
 8,
 9,
 10,
 0,
 9,
 8,
 9,
 0,
 6,
 7,
 8,
 8,
 7,
 8,
 0,
 10,
 0,
 9,
 6,
 9,
 10,
 7,
 0,
 0,
 0,
 9,
 8,
 8,
 7,
 5,
 2,
 0,
 9,
 9,
 8,
 10,
 8,
 0,
 8,
 6,
 8,
 8,
 7,
 10,
 9,
 7,
 6,
 0,
 6,
 0,
 5,
 0,
 0,
 0,
 5,
 8,
 6,
 0,
 8,
 8,
 9,
 8,
 8,
 9,
 0,
 0,
 0,
 6,
 0,
 8,
 0,
 10,
 5,
 7,
 0,
 

In [ ]:
preds

[2.3333333333333335,
 0.0,
 5.333333333333333,
 0.0,
 5.0,
 3.0,
 8.333333333333334,
 6.0,
 7.666666666666667,
 5.666666666666667,
 9.666666666666666,
 3.3333333333333335,
 5.666666666666667,
 4.666666666666667,
 5.0,
 5.0,
 5.666666666666667,
 8.0,
 2.6666666666666665,
 0.0,
 5.666666666666667,
 2.6666666666666665,
 5.666666666666667,
 7.333333333333333,
 0.0,
 1.6666666666666667,
 0.0,
 2.3333333333333335,
 3.3333333333333335,
 2.0,
 3.3333333333333335,
 2.3333333333333335,
 2.6666666666666665,
 5.666666666666667,
 6.0,
 4.666666666666667,
 8.666666666666666,
 2.6666666666666665,
 2.3333333333333335,
 7.0,
 8.0,
 2.3333333333333335,
 7.333333333333333,
 6.333333333333333,
 5.0,
 2.0,
 4.333333333333333,
 2.3333333333333335,
 9.333333333333334,
 0.0,
 0.0,
 5.0,
 4.666666666666667,
 2.6666666666666665,
 0.0,
 1.3333333333333333,
 3.3333333333333335,
 1.6666666666666667,
 0.0,
 4.0,
 4.666666666666667,
 6.0,
 5.666666666666667,
 1.6666666666666667,
 0.6666666666666666,
 3.0,
 7.3333333

In [ ]:
(ratings[484] + ratings[708] + ratings[2102])  / 3

5.0